In [6]:
conda install xlrd 

Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /Users/shreyashtiwari/opt/anaconda3

  added / updated specs:
    - xlrd


The following NEW packages will be INSTALLED:

  xlrd               pkgs/main/noarch::xlrd-2.0.1-pyhd3eb1b0_1 

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge/noarch::certifi-2024.8.30~ --> pkgs/main/osx-64::certifi-2024.8.30-py39hecd8cb5_0 
  conda              conda-forge::conda-24.9.2-py39h6e9494~ --> pkgs/main::conda-24.9.2-py39hecd8cb5_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd 
import numpy as np 

In [3]:
df = pd.read_excel("us-violence-brief-1.xls")

df.head()

,year,month,day,latitude,longitude,fatalities
0,2020,1,27,38.8137,-76.9453,1
1,2020,1,28,40.0444,-111.7320,0
2,2020,1,31,30.3674,-89.0928,1
3,2020,2,1,33.9375,-117.2310,1
4,2020,2,4,35.0887,-92.4421,1


In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

#training (2020-2022) and testing (2023)
train_data = df[df['year'] < 2023]
test_data = df[df['year'] == 2023]

# Separate features (latitude, longitude) and target (fatalities)
X_train = train_data[['latitude', 'longitude']]
y_train = train_data['fatalities']
X_test = test_data[['latitude', 'longitude']]
y_test = test_data['fatalities'] 


In [11]:
# Initialize the KNN model with k=3
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=3)

In [13]:
# Predict fatalities for 2023 test data
y_pred = knn.predict(X_test)

# Check if actual values for 2023 are available
if not y_test.empty:
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['No Fatality', 'Fatality'])
    print(f"Accuracy: {accuracy}")
    print("Classification Report:\n", report)
else:
    print("Predicted fatalities for 2023:\n", y_pred)


Accuracy: 0.8294930875576036
Classification Report:
               precision    recall  f1-score   support

 No Fatality       0.83      0.99      0.91       181
    Fatality       0.00      0.00      0.00        36

    accuracy                           0.83       217
   macro avg       0.42      0.50      0.45       217
weighted avg       0.70      0.83      0.76       217



** Approach 2 **

In [15]:
from sklearn.preprocessing import StandardScaler

# Create a temporal feature by combining year, month, day, latitude, and longitude
df['temporal_feature'] = df['year'] * 365 + df['month'] * 30 + df['day']  # day count
X_features = df[['latitude', 'longitude', 'temporal_feature']]

# Normalize the features to give equal importance to space and time
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)

# Update DataFrame with scaled features
df[['latitude_scaled', 'longitude_scaled', 'temporal_scaled']] = X_scaled

# Training and testing data 
train_data = df[df['year'] < 2023]
test_data = df[df['year'] == 2023]

X_train = train_data[['latitude_scaled', 'longitude_scaled', 'temporal_scaled']]
y_train = train_data['fatalities']
X_test = test_data[['latitude_scaled', 'longitude_scaled', 'temporal_scaled']]
y_test = test_data['fatalities'] 


In [23]:
#Initialize KNN with k=3
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=3)

In [25]:
# Predict fatalities for the 2023 test data
y_pred = knn.predict(X_test)

if not y_test.empty:
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['No Fatality', 'Fatality'])
    print(f"Accuracy: {accuracy}")
    print("Classification Report:\n", report)
else:
    print("Predicted fatalities for 2023:\n", y_pred)


Accuracy: 0.847926267281106
Classification Report:
               precision    recall  f1-score   support

 No Fatality       0.85      1.00      0.92       181
    Fatality       1.00      0.08      0.15        36

    accuracy                           0.85       217
   macro avg       0.92      0.54      0.54       217
weighted avg       0.87      0.85      0.79       217

